In [ ]:
library(ggplot2)
library(Seurat)
library(future)
library(data.table)
library(readxl)
library(plyr)

library(dplyr)
library(extrafont)
library(reshape2)

library(viridis)
library(knitr)
library(stringr)
library(NMF)
library(rsvd)
library(lme4)
library(RColorBrewer)

library(gtools)
library(gprofiler2)
options(future.globals.maxSize  = 8000*1024^2)

In [ ]:
datasets <- readRDS(".../RData/SEZ_seurat_object.rds")
DefaultAssay(datasets) <- "RNA"

In [ ]:
ast<-subset(datasets,ident=c('16','0','7','8',
                            
                            '19',
                            '11','13','14',
                            '2','3','12','10',
                            '9'
                            
                            ))
droplevels(ast)
levels(ast)

In [ ]:
DefaultAssay(ast) <- 'RNA'
ast[['integrated']]<- NULL
ast <- SplitObject(ast, split.by = 'donor_ID')

print(length(ast))
print('start find variable features.')
gc()
for (i in 1:length(ast)) {
    print(i)
    ast[[i]] <- NormalizeData(ast[[i]], verbose = FALSE)
    ast[[i]] <- FindVariableFeatures(ast[[i]], selection.method = "vst",  nfeatures = 2000, verbose = TRUE)
}

#features <- SelectIntegrationFeatures(object.list = ast)
# do CCA-based integration
gc()
anchors <- FindIntegrationAnchors(object.list = ast, 
                                  anchor.features = 2000, 
                                  dims = 1:30, reduction = 'cca', normalization.method = "LogNormalize")

saveRDS(anchors, file =  "anchors_NSCs_neurons.rds")
gc()

In [ ]:
# integrate

anchors <- readRDS("anchors_NSCs_neurons.rds")
ast <- IntegrateData(anchorset = anchors, dims = 1:30, 
                                 normalization.method = "LogNormalize", k.weight = 100)

DefaultAssay(ast) <- 'integrated'

print('done with integration, saving object..')


DefaultAssay(ast) <- 'integrated'
saveRDS(ast, file = "NSCs_neurons_CCA.rds")

# check if there are no NAs, sanity check
print(' number of NAs in integrated data' )
sum(is.na(ast[["integrated"]]@data))

# new clustering
print('scale')
ast <- ScaleData(object = ast, vars.to.regress = c("nCount_RNA", "percent.mito", "percent.ribo","nFeature_RNA"))
saveRDS(ast, file = "NSCs_neurons_CCA_scaled.rds")

print('PCA')

ast <- RunPCA(ast, npcs = 50)
ElbowPlot(ast, ndims = 50)
print('check ndims')

saveRDS(ast, file = "NSCs_neurons_CCA_scaled.rds")